In [ ]:
import jupyterlab_nodeeditor as jlne
import ipywidgets, yaml
from yggdrasil.runner import YggRunner
from model_loader import dict_conversion, load_model

In [ ]:
# Here is where we will use the parser
# Start by opening it up
with open("yggdrasil_test_models/model_trifecta.yml") as fd:
    tester = yaml.safe_load(fd)

In [ ]:
# This will be a cleaner for the sidebar display
def clean_bytes(ibytes):
    sbytes = str(ibytes)[1:].strip("'")
    if "\\n" in sbytes:
        clean = sbytes[:-2]
    else:
        clean = sbytes
    return clean

In [ ]:
# Initialize the node editor we are going to use 
ne = jlne.NodeEditor()
# Use yggdrasil_test_models/model_trifecta.yml or wherever the trifecta model is loaded locally for the example
load_model(ne)

In [ ]:
# Create the callback class and have the third component be updated with live information
# As of March 30th, 2022 there is an issue with connected components in JLNE but the third tab will display the correct information regardless
class CBJLNE:
    if len(ne.node_editor.nodes[2].display_element.children) <= 5:
        for i in range(3):
            ne.node_editor.nodes[2].display_element.children += (ipywidgets.Label("Initializing"),)
    
    def __init__(self):
        self.label = ipywidgets.Label()
    
    def display_number(self, *args, **kwargs):
        ne.node_editor.nodes[2].display_element.children[3].value = f"Number: {args[0].args[0]}"
        
    def display_string(self, *args, **kwargs):
        print(clean_bytes(args[0].args))
        ne.node_editor.nodes[2].display_element.children[4].value = f"String: {clean_bytes(args[0].args)}"
        
    def display_rotated(self, *args, **kwargs):
        ne.node_editor.nodes[2].display_element.children[5].value = f"Rotated: {args[0].args}"
    
display = CBJLNE()

In [ ]:
# Create a runner for the model, adjust the filepath for wherever the model is located
runner = YggRunner("yggdrasil_test_models/model_trifecta.yml")

In [ ]:
# Add in the callback functions to the connection drivers for execution
runner.connectiondrivers['model_reader:inputReader']['callbacks'] = [display.display_string]
runner.connectiondrivers['model_rng:outputRNG,model_reader:outputReader_to_model_rotxx:input_rotxx']['callbacks'] = [display.display_number]
runner.connectiondrivers['model_rotxx:output_rotxx']['callbacks'] = [display.display_rotated]

In [ ]:
# Run the runner to see the data update in realtime
runner.run()